In [1]:
x -> x

#3 (generic function with 1 method)

In [6]:
((x,y)->x+y)(1,2)

3

In [4]:
(x -> y -> x)(1)(2)

1

In [9]:
factorial(3)

6

In [12]:
function sillysin(err,x)
    t = 1
    y = 0
    n = 1
    x = x % (2 * pi)
    while t >= err
      t = (-x)^n / factorial(n) 
      y -= t 
      n += 2
    end
    return y
end

sillysin (generic function with 1 method)

In [ ]:
sillysin(0.3, )

In [17]:


function square(eps)
    return sqrt(eps) , x -> x * x 
end

function add(eps1,eps2)
    return min(eps1,eps2) / 2, (x,y) -> x + y
end    

function sin(eps)
    return eps/2, x -> sintoaccuracy(eps/2,x)
end
        
function lenscompose(f,g)
    function res(epsz)
        epsy, f2 = f(epsz)
        epsz, g2 = g(epsy)
        return epsz, x -> f2(g2(x))
    end
    return res
end


lenscompose (generic function with 1 method)

The exact reals seem like an interesting place for 

If we make the precision required static and the actual computation 
https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/#Arbitrary-Precision-Arithmetic-1

Julia contains gnu multiprecision by default. That's rpetty cool


Wait, since having a derivative is a stronger condition than continuity, I ought to be able to directly take reverse mode and get verfied bounds

No, I need a symbolic derivative + interval analysis of the derivative.

Maybe I want a automatic differentiation applied to intervals. 


Why do I want to do it in stages?
Well, then we spend more precision on parts that matter and less on parts that don't



Is there any sense in which this is better than interval airthemtic itself?
Interval arithemtic does not arbitrarily converge by construction? But it probably does...

Sometimes you care about a particular output precision.

Pre analysis let's you use "cheaper?" computations. not sure mpfi is that much cheaper.


So there's a forward mode interval analysis
backward mode precision analysis
and then a forward mode multiprecision analysis.


lazy boolean

T, F, bot.
what is dx
x > 7
`>` takes two numbers. 7 is known exactly.to guarantee T/F/bot is known, we have to 

filters? the forward interval analysis will tell us if T, F is possible.

https://members.loria.fr/PZimmermann/irram18.pdf
Hmm this is explcilty mentioned DAGs. 
"Bottomn up and top down"
Yeah. The computation graph he's describing is very similar to differentiation
I think I'm describing something similar to "top down"


In [ ]:
BigFloat(1)

In [ ]:
lens( dy ) = (dy , x::Expr -> x)  

compose(l1,l2) = dz -> let (dy,  f1 ) = l1 dz in 
                       let (dx , f2 ) = l2 dy in   
                       (dx, x -> :(begin
                                    
                                   q = $(f1 x)
                                   setprecision( $(dy) )
                                   let z = $(f2 :(q)) in
                                   z
                                   
                    
                                     end))       
                       end 
                       end
    
    
    

In [33]:

prec = 10
f1 = q -> q * q
f2 = q -> sqrt(q)
# we may want to cps this so we can put the lets in good places.
ex = x -> :(begin
         let y = $(f1)($(x))
         setprecision($(prec))
         let z = $(f2)(y) # or just avoid the let here
            z
            end
         end
        end)

sealex = eval(:(x -> $( ex(:(x)) ) ))

#89 (generic function with 1 method)

In [40]:
@code_typed sealex(BigFloat(7))
# code_lowered code_llvm code_native

CodeInfo(
1 ─ %1 = invoke Main.:*(_2::BigFloat, _2::BigFloat)::BigFloat
│   %2 = Base.MPFR.DEFAULT_PRECISION::Base.RefValue{Int64}
│        Base.setfield!(%2, :x, 10)::Int64
│   %4 = invoke Main.sqrt(%1::BigFloat)::BigFloat
└──      return %4
) => BigFloat

In [41]:
@code_lowered sealex(BigFloat(7))

CodeInfo(
1 ─     y = (var"#83#84"())(x)
│       Main.setprecision(10)
│       z = (var"#85#86"())(y)
└──     return z
)

In [23]:
sealex(BigFloat(7))

MethodError: MethodError: no method matching (::var"#37#38")(::BigFloat)
Closest candidates are:
  #37(!Matched::Expr) at In[21]:6

In [43]:

prec = 10
f1 = q -> q * q
f2 = q -> sqrt(q)
# we may want to cps this so we can put the lets in good places.
ex = x -> begin
          let y = f1(x)
          setprecision(prec)
          let z = f2(y) # or just avoid the let here
          z
            end
         end
        end


#100 (generic function with 1 method)

In [44]:
@code_lowered ex(BigFloat(7))

CodeInfo(
1 ─     y = Main.f1(x)
│       Main.setprecision(Main.prec)
│       z = Main.f2(y)
└──     return z
)

In [2]:
using ForwardDiff

In [3]:
using IntervalArithmetic

In [52]:
ForwardDiff.derivative(sin, Float64(π) )

-1.0

In [57]:
ForwardDiff.derivative(sin, @biginterval(1.0))

[0.540302, 0.540303]₂₅₆

In [58]:
ForwardDiff.derivative(sin, @interval(1.0))

[0.540302, 0.540303]

In [64]:
# evaluating x in the interval [0,1], in order to get +- 0.1 accuracy on output, we need to evaluate with +- 0.185 accuracy
# I haven't thoguht through this right.
@interval(-0.1,0.1) / ForwardDiff.derivative(sin, @interval(0,1))

[-0.185082, 0.185082]

In [ ]:
struct Reverseo
    domaino
    k
end


Base.:+(x::Reverseo , y::Reverseo) =
      let df = max( abs(ForwardDiff.gradient( x -> x[1] + x[2] , [  x.domaino , y.domaino ]  )))
      let epsz -> (epsz / df[1] / 2 , epsz / df[2] / 2   )
        

end

    Base.sum(x::Reverseo ) = x.domaino yada
                             Reverso( sum(x.domaino)  , epsz ->  x.k ( epsz  /  df   ,  x2 -> setprecision(epsz) ; sum(x2) ) 
    

::Reverseo * ::Reverseo



In [1]:
struct Foo end

x::Foo + y::Foo = Foo()

+ (generic function with 1 method)

In [2]:
Foo() + Foo()

Foo()

In [6]:
Base.sin(x::Foo) = Foo()

In [7]:
sin(Foo())

Foo()

In [8]:
methods(sin)

# 1 method for generic function "sin":
[1] sin(x::Foo) in Main at In[3]:1

(x -> (y, dy -> dx) ~ (x -> (y, forall s. (dx -> s) -> (dy -> s)) ~> forall s. (x , dx -> s) -> (y , dy -> s)

What this is expressing is that reverse mode AD is a map on linear functionals


DualIx -> (DualIy, (epsy -> (epsx, (x -> y)))

(DualIx, epsx -> s)

epsx , (y -> s) -> (x -> s)
  (epsy, y -> s) -> (epsx, x -> s)
    (DualIx , ((epsx, x -> s) -> t))  -> (DualIy, ((epsy, y -> s) -> t))
    DualIx -> ((epsx, x -> s) -> t))  -> (DualIy, ((epsy, y -> s) -> t)) 
    
    
run(f, initdom, epsy) = let Iy, k = f(InitDom, id)   # t is now (epsx, x->s) so maybe I want snd? or curry it?
                        let f2  = k (epsy, id)
                        in  x::BigFloat -> f2(x)
   
   
struct ExactNum 
q :: DualIx, (epsx, x -> s) -> t
end


cos(sin(ExactNum))




In [1]:


struct Stage2
    eps
    k # x -> s
end

struct Stage1
   inter 
   k # Stage2 -> 
end

In [69]:
Base.sin(x :: Stage1) = 
        Stage1( sin(x.inter) , 
        epsyk :: Stage2 ->
              begin
              println(abs(ForwardDiff.derivative(sin, x.inter )).hi)
              epsx = epsyk.eps / abs(ForwardDiff.derivative(sin, x.inter )).hi  #  we should allow epx to be inifnity
              println(epsx)
              x.k( Stage2( epsx , 
                           let prec = max(2, Int64(ceil(-log2(epsx))))
                           fpx ::BigFloat ->
                           begin
                           println(prec)
                           println(max(2,prec))
                           setprecision(prec) # round up?
                           epsyk.k(sin(fpx))
                           end
                           end
                           )
                 )
              end
         )

In [100]:
Base.sin(x :: Stage1) = 
        Stage1( sin(x.inter) , 
        epsyk :: Stage2 ->
              begin
              println(abs(ForwardDiff.derivative(sin, x.inter )).hi)
              epsx = epsyk.eps / abs(ForwardDiff.derivative(sin, x.inter )).hi  #  we should allow epx to be inifnity
              println(epsx)
              x.k( Stage2( epsx , 
                           let prec = max(2, Int64(ceil(-log2(epsx))))
                           fpx  ->
                           :( begin
                              setprecision( $prec ) # round up?
                              q =  sin( $fpx )
                              $(epsyk.k( :q ))
                              end
                           )
                           end
                           )
                 )
              end
         )

I have not done this right. Precision is relative precision isn't it
epsx * df = epsy
log2( epsy / y )

ln(f(x))


In [110]:
setprecision(4)
BigFloat(21000000)

2.1e+07

In [99]:
k = sin
:(x -> $(  ))

:(x->begin
          #= In[99]:2 =#
          ()
      end)

In [101]:
sy1 =sin(sin(sin(sin(Stage1(  Interval(0,0.4)  , sy1 -> sy1 )))))
sx2 = sy1.k( Stage2(0.0001,  sx2 -> sx2 ))
prec = max(2, Int64(ceil(-log2(sx2.eps))))
setprecision(prec)
exactresulttoprecpointone = :(x -> $(sx2.k( :x ))) # also inspect this code


1.0
0.0001
1.0
0.0001
1.0
0.0001
1.0
0.0001


:(x->begin
          #= In[101]:5 =#
          begin
              #= In[100]:12 =#
              setprecision(14)
              #= In[100]:13 =#
              q = sin(x)
              #= In[100]:14 =#
              begin
                  #= In[100]:12 =#
                  setprecision(14)
                  #= In[100]:13 =#
                  q = sin(q)
                  #= In[100]:14 =#
                  begin
                      #= In[100]:12 =#
                      setprecision(14)
                      #= In[100]:13 =#
                      q = sin(q)
                      #= In[100]:14 =#
                      begin
                          #= In[100]:12 =#
                          setprecision(14)
                          #= In[100]:13 =#
                          q = sin(q)
                          #= In[100]:14 =#
                          q
                      end
                  end
              end
          end
      end)

In [71]:
sy1 =sin(sin(sin(sin(Stage1(  Interval(0,0.4)  , sy1 -> sy1 )))))
sx2 = sy1.k( Stage2(0.0001,  sx2 -> sx2 ))
prec = max(2, Int64(ceil(-log2(sx2.eps))))
setprecision(prec)
exactresulttoprecpointone = sx2.k( BigFloat(0.3) ) # also inspect this code
#k = sx2.k
@code_lowered sx2.k(BigFloat(0.3) )

1.0
0.0001
1.0
0.0001
1.0
0.0001
1.0
0.0001
14
14
14
14
14
14
14
14


CodeInfo(
1 ─ %1  = Core.getfield(#self#, :prec)
│         Main.println(%1)
│   %3  = Core.getfield(#self#, :prec)
│   %4  = Main.max(2, %3)
│         Main.println(%4)
│   %6  = Core.getfield(#self#, :prec)
│         Main.setprecision(%6)
│   %8  = Core.getfield(#self#, :epsyk)
│   %9  = Base.getproperty(%8, :k)
│   %10 = Main.sin(fpx)
│   %11 = (%9)(%10)
└──       return %11
)

In [72]:
sin(Interval(0,0.4))

[0, 0.389405]

In [59]:
Int64(ceil(-log2(0.01)))

7

In [67]:
sin(sin(sin(sin(0.3))))

0.28320844515030463

In [14]:
Int64(ceil(log2(2)))

UndefVarError: UndefVarError: ceiling not defined

In [15]:
ceil(1.8)

2.0